In [1]:
import pyaccel
import pymodels
import numpy as np
import matplotlib.pyplot as plt
from mathphys.functions import save_pickle, load_pickle

import touschek_pack.functions as tousfunc
import touschek_pack.Classes as tousclass


In [2]:
# criando o modelo para a realização do tracking
acc = pymodels.si.create_accelerator()
acc.cavity_on = True
acc.radiation_on = True
acc.vchamber_on = True

In [44]:


# é bom lembrar também que os scrapers que eu estou interessado estão disponíveis no modelo como sendo SVVC e SHVC
# designando set vertical caccum chamber and set horizontal vaccum chamber

# defining the nlk position to track the electrons
dicty = pyaccel.lattice.find_dict(acc, 'fam_name')
nlk_index = dicty['InjNLKckr'][0]
spos = pyaccel.lattice.find_spos(acc, indices='closed')

s_nlk = float(spos[nlk_index])


In [4]:
# definindo também o modelo fitado para calcular a distribuição de equilíbrio do feixe

fit_acc = pymodels.si.create_accelerator()
fit_acc = pymodels.si.fitted_models.vertical_dispersion_and_coupling(fit_acc)
tousan = tousclass.Tous_analysis(fit_acc)
deltas = tousan.deltas
n_turns = tousan.nturns

In [11]:
# como podemos criar uma função que varie os parametros da forma que eu quero ?
# eu preciso variar as posições x e y dos elétrons no trecho de injeção e também variar os angulos x e y
# eu poss simplesmente criar um linspace com as variações de parâmetros que eu desejo



(460,)

In [ ]:
# agora nesse estágio eu preciso fazer variar as posições dos 
tousfunc.track_eletrons(deltas, n_turns, nlk_index, )

In [13]:
orb = pyaccel.tracking.find_orbit6(acc, indices=[0, nlk_index])
orb = orb[:, 1]

In [17]:
np.zeros((6, deltas.size)), orb[:, None]

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 array([[-2.25702434e-08],
        [-4.83326693e-09],
        [ 0.00000000e+00],
        [ 0.00000000e+00],
        [-6.34753037e-05],
        [-1.51791706e-02]]))

In [18]:
rin = np.zeros((6, deltas.size))

rin += orb[:,None]  

In [19]:
rin

array([[-2.25702434e-08, -2.25702434e-08, -2.25702434e-08, ...,
        -2.25702434e-08, -2.25702434e-08, -2.25702434e-08],
       [-4.83326693e-09, -4.83326693e-09, -4.83326693e-09, ...,
        -4.83326693e-09, -4.83326693e-09, -4.83326693e-09],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-6.34753037e-05, -6.34753037e-05, -6.34753037e-05, ...,
        -6.34753037e-05, -6.34753037e-05, -6.34753037e-05],
       [-1.51791706e-02, -1.51791706e-02, -1.51791706e-02, ...,
        -1.51791706e-02, -1.51791706e-02, -1.51791706e-02]])

In [26]:
import pyaccel.optics as pyop

mean = np.zeros(6)
env = pyop.beam_envelope.calc_beamenvelope(fit_acc)

cov = env[nlk_index]


In [41]:
# Dessa forma uma distribuição gaussiana do feixe é criada
# a partir disso eu posso ir variando cada valor das posições tanto em x quanto em y, além de variar também os angulos x' e y'
# mas como eu posso variar esses parâmetros ?

particles = np.random.multivariate_normal(mean, cov, size=5000).T


In [46]:
particles[0]

array([-1.64886840e-05, -5.51643546e-05,  1.43536208e-05, ...,
       -2.69156919e-05,  1.25715526e-05,  9.87212800e-06])

In [43]:
# # célula colocada aqui simplesmente para teste de verificação
# # ao executar o código abaixo é possível observar a distribuição gaussiana das posições em x
# plt.figure()
# plt.hist(particles[0], bins=200)
# plt.show()